# OPM Personnel Data and USAJobs API

#### Things to do going forward:

#### read data from opm website (CHECK)

#### generalize the cosine sim function (CHECK)

#### clean up the state mapping function

#### better process USAJobs data

### Reading Packages

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import date, datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer
from bokeh.plotting import figure, show, output_file,  output_notebook
from bokeh.sampledata.us_states import data as states
import requests, zipfile
import io
pd.set_option('mode.chained_assignment', None)

### Get Directory

In [2]:
#myDir=r"H:\_MyComputer\Documents\Git Repos\myOpmData\main\personnelData"
myDir=r"C:\Users\admin\Documents\Python Scripts\personnel\personnelData"
os.chdir(myDir)

### Get Relevant Business Occupations

In [3]:
dfOccs=pd.read_excel(r"C:\Users\admin\Documents\Python Scripts\personnel\personnelData\Scripts\EBO Occupation Code Mapping.xlsx", dtype=str)
#this cleans the data to get the four-digit occupation code
def addLeadingZero(mystring):
    if len(mystring)==3:
        return("0" + mystring)
    else:
        return(mystring)

dfOccs['Occupation Code']=dfOccs['Occupation Code'].apply(addLeadingZero)
len(dfOccs['Occupation Code'].unique())

46

### Functions

In [4]:
def makeDF(fileName, dictOfDF):
    dfName=fileName.split(".")[0]
    df=pd.read_csv(fileName, low_memory=False)
    dictOfDF[dfName]=df
    #print(dfName, df.columns)
    return(dictOfDF)

def subsetDF(df):
    subsetDF=df.loc[df['AGY']=="DD"]
    return(subsetDF)

def mergeSubset(mergeTable, allTable):
    mergedData=mergeTable.merge(allTable, left_on='AGYSUB', right_on='AGYSUB')
    return(mergedData)

def genericMerge(df1, df2):
    mergeColumns=[i for i in df1.columns if i in df2.columns]
    dfMerge=df1.merge(df2, left_on=mergeColumns, right_on=mergeColumns, how='left')
    return(dfMerge)

def tellMeAboutIt(df,agency):
    agencySubset=df.loc[df['AGYSUBT']==agency]
    topOccs=(agencySubset['OCCT'].value_counts(normalize=True).head(1))
    return(topOccs)

def occsByPercent(df,agency):
    agencySubset=df.loc[df['AGYSUBT']==agency]
    allOccs=(agencySubset['OCCT'].value_counts(normalize=True))
    return(allOccs)
    
def makeWordTemplate(df, AgencyName):
    document = Document()
    document.add_heading(f'Civilian Personnel Fact Sheet: {AgencyName}', level=1)
    document.add_heading(f"Average Salary: {'${:,.0f}'.format(df['SALARY'].mean())}", level=2)
    document.add_heading(f"Number of Employees: {'{:,.0f}'.format(len(df))}", level=2)
    #document.add_heading(f"Top Occupations: {'{:,.0f}'.format(len(df))}", level=2)
    data=getTopValuesPercents(df['OCCT'])
    #document=doctable(data, document)
    document.save('demo.docx')

def getTopValuesPercents(dfAndColumn, myNumber=5):
    seriesNumbers=dfAndColumn.value_counts().head(myNumber).map('{:,.0f}'.format)
    seriesPercents=dfAndColumn.value_counts(normalize=True).head(myNumber).map("{:.0%}".format)
    dfSum=pd.concat([seriesNumbers, seriesPercents], axis=1).reset_index()
    dfSum.columns=["Occupation", "Count", "Percent"]
    return(dfSum)

def doctable(data, document):
    table = document.add_table(rows=(data.shape[0]+1), cols=data.shape[1])
    for i, column in enumerate(data):
        for row in range(data.shape[0]-1) :
            table.cell(row, i).text = str(data[column][row])
    return(document)

def relevantJobs(wordsToFind, df):
    listOfJobs=[]
    for word in wordsToFind:
        listOfJobs.append([i for i in df['OCCT'].unique() if word in i.lower()])
    flat_list = [item for sublist in listOfJobs for item in sublist]
    allJobs=list(set(flat_list))
    return(allJobs)
        
def jobsByAgency(df, allJobs):
    df['selectedJobs']=0
    df.loc[df['OCCT'].isin(allJobs),'selectedJobs']=1
    allJobsByAgency=df[['AGYSUBT','selectedJobs']].groupby(['AGYSUBT']).agg(['mean', 'sum'])['selectedJobs'].sort_values(['sum'], ascending=False)
    return(allJobsByAgency.loc[allJobsByAgency['sum']>0])

def getRelJobsByAgency(wordsToFind, df):
    allJobs=relevantJobs(wordsToFind, df)
    allJobsByAgency=jobsByAgency(df, allJobs)
    return(allJobs, allJobsByAgency)

def getAllOccsbyAgency(df, cutoff):
    agencies=list(df['AGYSUBT'].unique())
    listOfAllOccs=[]
    for agency in agencies:
        results=occsByPercent(df,agency)
        listOfAllOccs.append([agency, results.index[0], results.values[0]])
    allOccs=pd.DataFrame(listOfAllOccs, columns=["Agency", "Occupation", "Percent"]).set_index("Agency")
    cutoffOccs=allOccs.loc[allOccs['Percent']>=cutoff]
    cutoffOccs['Percent']=cutoffOccs['Percent'].map("{:.0%}".format)
    return(cutoffOccs)

def occsByPercentOverall(df,occ):
    occSubset=df.loc[df['OCCT']==occ]
    allAgencies=(occSubset['AGYSUBT'].value_counts(normalize=True))
    return(allAgencies)

def getAllAgenciesByOcc(df, cutoff):
    occs=list(df['OCCT'].unique())
    listOfAllAgencies=[]
    for occ in occs:
        results=occsByPercentOverall(df,occ)
        listOfAllAgencies.append([occ, results.index[0], results.values[0]])
    allAgencies=pd.DataFrame(listOfAllAgencies, columns=["Occupation", "Agency", "Percent"]).set_index("Occupation")
    cutoffAgencies=allAgencies.loc[allAgencies['Percent']>=cutoff]
    cutoffAgencies['Percent']=cutoffAgencies['Percent'].map("{:.0%}".format)
    return(cutoffAgencies)

def overlapTwoSpecializedMeasures(results1, results2):
    overlap=results1.reset_index().merge(results2.reset_index(), right_on=['Occupation', 'Agency'], left_on=['Occupation', 'Agency'])
    overlap.columns=['Agency', 'Occupation', 'Percent at Agency', 'Percent Overall']
    return(overlap)

def putTogetherSpecializationOverlap(df, cutoffAtAgency, cutoffsOverAllAgencies):
    results1=getAllOccsbyAgency(ddDataCopy,cutoffAtAgency)
    results2=getAllAgenciesByOcc(ddDataCopy,cutoffsOverAllAgencies)
    overlap=overlapTwoSpecializedMeasures(results1, results2)
    return(overlap)

def get_cosine_sim(*strs): 
    vectors = [t for t in get_vectors(*strs)]
    return cosine_similarity(vectors)

def keepNumbers(myString):
    justNumbers=''.join(i for i in myString if i.isdigit())
    return(justNumbers)

def connect():
    """ Forms the header parameters for authenticating with the v2 api. """
    headers = {'Authorization-Key': 'zm3i+Zu2AiT6+w3KqawsnwHaPkYexPVNaDJgYzcfWNc=',
 'Host': 'data.usajobs.gov',
 'User-Agent': 'abigail.haddad@gmail.com'}
    return headers
        
def format_dict(results, df_fixed):
    for j in range(0, len(results['SearchResult']['SearchResultItems'])):
        df_fixed=df_fixed.append(pd.DataFrame.from_dict(results['SearchResult']['SearchResultItems'][j]['MatchedObjectDescriptor'], orient='index').transpose())
        df_fixed=df_fixed.apply(pd.Series)
    return df_fixed

def current_search(agencyNum):
    """ Constructs v3 fuzzy searches with parameters outlined here:
    http://search.digitalgov.gov/developer/jobs.html """
    number=str(1)
    strAgencyNum=str(agencyNum)
    #base_url=f"https://data.usajobs.gov/api/search?Keyword={terms}&WhoMayApply=All&p="
    base_url=f"https://data.usajobs.gov/api/search?Organization=DD{strAgencyNum}&p="
    df_fixed=pd.DataFrame()
    results = requests.get(base_url+number, headers=connect()).json()
    #print(base_url+number)
    df_fixed=format_dict(results, df_fixed)
    if len(results['SearchResult']['SearchResultItems']) == 10:
            while results['SearchResult']['SearchResultCount']!= 0 and int(number)<101:
                number=str(int(number)+1)      
                results = requests.get(base_url+number, headers=connect()).json()
                #print(base_url+number)
                df_fixed=format_dict(results, df_fixed)
    return df_fixed

def pullAndUnzip(zipLink):
    r = requests.get(zipLink)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()
    
def cosinSimAcrossAgencies(df, variable):
    intermedDF=df[['AGYSUBT', variable, 'AGYTYP']].groupby(['AGYSUBT', variable]).count()
    intermedPct=intermedDF.groupby(level=0).apply(lambda x: 100 * x / float(x.sum())).reset_index()
    intermedPct.columns=['AGYSUBT', variable, 'PERCENT']
    reshapedDF=intermedPct.pivot(index='AGYSUBT', columns=variable, values='PERCENT')
    reshapedDF=reshapedDF.fillna(0) 
    listOfLists=[]
    for rowNumber1 in range(0, len(reshapedDF)):
        for rowNumber2 in range(0, len(reshapedDF)):
            if rowNumber1>rowNumber2:
                listToAppend=[reshapedDF.index[rowNumber1], reshapedDF.index[rowNumber2],cosine_similarity([reshapedDF.iloc[rowNumber1],reshapedDF.iloc[rowNumber2]])[0][1] ]
                listOfLists.append(listToAppend)
    dfCosine = pd.DataFrame(listOfLists)
    dfCosine.columns=['Org1', 'Org2', 'Similarity Index']
    dfCosine=dfCosine.sort_values('Similarity Index', ascending=False)
    return(dfCosine)


### Download OPM Files

In [5]:
pullAndUnzip("https://www.opm.gov/Data/Files/543/b8c3885d-a8e2-43ce-83f4-0398a1dfbc33.zip")    

### Read In OPM Files

In [6]:
textFiles=[i for i in os.listdir() if i.lower().endswith(".txt")]
dictOfDF={}
for fileName in textFiles:
    dictOfDF=makeDF(fileName, dictOfDF)

### Subset Just Defense Department Data

In [7]:
ddAgencies=subsetDF(dictOfDF['DTagy'])
ddData= mergeSubset(ddAgencies, dictOfDF['FACTDATA_MAR2018'])

### Join With All Lookup Tables

In [8]:
ddDataCopy=ddData
textClean=[i.split(".")[0] for i in textFiles]
for fileName in textClean:
    ddDataCopy=genericMerge(ddDataCopy,dictOfDF[fileName])

### Look At Missing Data

In [9]:
for col in ddDataCopy.columns:
    count=ddDataCopy[col].count()
    if count<len(ddDataCopy):
        print(col, count/len(ddDataCopy))

EDLVL 0.9990813497596746
GSEGRD 0.6771108449941764
SALARY 0.9751718368083466


### Agency List

In [10]:
agencies=list(ddDataCopy['AGYSUBT'].unique())
for agency in agencies:
    print(agency)

DD01-OFFICE OF THE SECRETARY OF DEFENSE
DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF
DD04-DEFENSE INFORMATION SYSTEMS AGENCY
DD06-DEFENSE SECURITY COOPERATION AGENCY
DD07-DEFENSE LOGISTICS AGENCY
DD08-U.S. COURT OF APPEALS FOR THE ARMED FORCES
DD10-DEFENSE CONTRACT AUDIT AGENCY
DD12-DEFENSE SECURITY SERVICE
DD13-DEFENSE ADVANCED RESEARCH PROJECTS AGENCY
DD15-UNIFORMED SERVICES UNIVERSITY OF THE HEALTH SCIENCES
DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY
DD21-WASHINGTON HEADQUARTERS SERVICES
DD23-OFFICE OF ECONOMIC ADJUSTMENT
DD25-DEFENSE LEGAL SERVICES AGENCY
DD26-OFFICE OF THE INSPECTOR GENERAL
DD27-MISSILE DEFENSE AGENCY
DD29-DEFENSE TECHNOLOGY SECURITY ADMINISTRATION
DD34-DEFENSE COMMISSARY AGENCY
DD35-DEFENSE FINANCE AND ACCOUNTING SERVICE
DD48-DEFENSE HUMAN RESOURCES ACTIVITY
DD53-DEFENSE POW/MIA ACCOUNTING AGENCY
DD60-DEFENSE HEALTH AGENCY
DD61-DEFENSE THREAT REDUCTION AGENCY
DD63-DEFENSE CONTRACT MANAGEMENT AGENCY
DD65-PENTAGON FORCE PROTECTION AGENCY
DD68-DEPARTMENT OF DEFE

### Top Occupation by Agency

In [11]:
listsOfTopOccs=[]
for agency in agencies:
    results=tellMeAboutIt(ddDataCopy,agency)
    listsOfTopOccs.append([agency, results.index[0], results.values[0]])
    
listsOfTopOccs

topOccs=pd.DataFrame(listsOfTopOccs, columns=["Agency", "Top Occupation", "Percent"]).set_index("Agency")
topOccs['Percent']=topOccs['Percent'].map("{:.0%}".format)

In [12]:
topOccs.sort_values("Percent", ascending=False)

,Top Occupation,Percent
Agency,,
DD10-DEFENSE CONTRACT AUDIT AGENCY,0511-AUDITING,90%
DD48-DEFENSE HUMAN RESOURCES ACTIVITY,0301-MISCELLANEOUS ADMINISTRATION AND PROGRAM,80%
DD65-PENTAGON FORCE PROTECTION AGENCY,0083-POLICE,70%
DD25-DEFENSE LEGAL SERVICES AGENCY,0905-GENERAL ATTORNEY,70%
DD23-OFFICE OF ECONOMIC ADJUSTMENT,1101-GENERAL BUSINESS AND INDUSTRY,69%
DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY,1701-GENERAL EDUCATION AND TRAINING,59%
DD83-MILITARY TREATMENT FACILITIES UNDER DHA,0610-NURSE,57%
DD71-DEFENSE MICROELECTRONICS ACTIVITY,0855-ELECTRONICS ENGINEERING,55%
DD68-DEPARTMENT OF DEFENSE TEST RESOURCE MANAGEMENT CENTER,0801-GENERAL ENGINEERING,52%


### Who Has Auditors?

In [13]:
auditOutput=getRelJobsByAgency(["audit"], ddDataCopy)
print(auditOutput[0])
auditOutput[1]

['0511-AUDITING']


,mean,sum
AGYSUBT,,
DD10-DEFENSE CONTRACT AUDIT AGENCY,0.898468,4398
DD26-OFFICE OF THE INSPECTOR GENERAL,0.363582,613
DD35-DEFENSE FINANCE AND ACCOUNTING SERVICE,0.004608,58
DD07-DEFENSE LOGISTICS AGENCY,0.000799,21
DD04-DEFENSE INFORMATION SYSTEMS AGENCY,0.002317,14
DD27-MISSILE DEFENSE AGENCY,0.005567,14
DD34-DEFENSE COMMISSARY AGENCY,0.000592,8
DD63-DEFENSE CONTRACT MANAGEMENT AGENCY,0.000625,7
DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY,0.000110,2


### Who Has Accountants?

In [14]:
accountOutput=getRelJobsByAgency(["account"], ddDataCopy)
print(accountOutput[0])
accountOutput[1]

['0510-ACCOUNTING', '0525-ACCOUNTING TECHNICIAN']


,mean,sum
AGYSUBT,,
DD35-DEFENSE FINANCE AND ACCOUNTING SERVICE,0.435529,5482
DD07-DEFENSE LOGISTICS AGENCY,0.007001,184
DD34-DEFENSE COMMISSARY AGENCY,0.006069,82
DD04-DEFENSE INFORMATION SYSTEMS AGENCY,0.006455,39
DD01-OFFICE OF THE SECRETARY OF DEFENSE,0.012745,26
DD27-MISSILE DEFENSE AGENCY,0.009145,23
DD21-WASHINGTON HEADQUARTERS SERVICES,0.009066,20
DD60-DEFENSE HEALTH AGENCY,0.003436,20
DD61-DEFENSE THREAT REDUCTION AGENCY,0.014190,19


### Who Has Human Resources?

In [15]:
humanOutput=getRelJobsByAgency(["human"], ddDataCopy)
print(humanOutput[0])
humanOutput[1]

['0203-HUMAN RESOURCES ASSISTANCE', '0201-HUMAN RESOURCES MANAGEMENT', '0299-HUMAN RESOURCES MANAGEMENT STUDENT TRAINEE']


,mean,sum
AGYSUBT,,
DD07-DEFENSE LOGISTICS AGENCY,0.024503,644
DD48-DEFENSE HUMAN RESOURCES ACTIVITY,0.057789,309
DD35-DEFENSE FINANCE AND ACCOUNTING SERVICE,0.020338,256
DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY,0.005604,102
DD21-WASHINGTON HEADQUARTERS SERVICES,0.037625,83
DD34-DEFENSE COMMISSARY AGENCY,0.004218,57
DD63-DEFENSE CONTRACT MANAGEMENT AGENCY,0.004732,53
DD27-MISSILE DEFENSE AGENCY,0.019483,49
DD61-DEFENSE THREAT REDUCTION AGENCY,0.032114,43


### Measures of Agency Specialization in an Occupation

#### Occupations Which Are Owned by An Agency

In [16]:
putTogetherSpecializationOverlap(ddDataCopy, 0, .5)

,Agency,Occupation,Percent at Agency,Percent Overall
0,DD10-DEFENSE CONTRACT AUDIT AGENCY,0511-AUDITING,90%,86%
1,DD13-DEFENSE ADVANCED RESEARCH PROJECTS AGENCY,0850-ELECTRICAL ENGINEERING,15%,65%
2,DD15-UNIFORMED SERVICES UNIVERSITY OF THE HEAL...,0401-GENERAL NATURAL RESOURCES MANAGEMENT AND ...,30%,93%
3,DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY,1701-GENERAL EDUCATION AND TRAINING,59%,98%
4,DD34-DEFENSE COMMISSARY AGENCY,2091-SALES STORE CLERICAL,25%,100%
5,DD35-DEFENSE FINANCE AND ACCOUNTING SERVICE,0525-ACCOUNTING TECHNICIAN,24%,98%
6,DD60-DEFENSE HEALTH AGENCY,0610-NURSE,16%,94%
7,DD63-DEFENSE CONTRACT MANAGEMENT AGENCY,1910-QUALITY ASSURANCE,30%,78%
8,DD65-PENTAGON FORCE PROTECTION AGENCY,0083-POLICE,70%,73%
9,DD80-DEFENSE MEDIA ACTIVITY,1071-AUDIOVISUAL PRODUCTION,18%,70%


#### Occupations Which Dominate an Agency

In [17]:
putTogetherSpecializationOverlap(ddDataCopy, .5, 0)

,Agency,Occupation,Percent at Agency,Percent Overall
0,DD10-DEFENSE CONTRACT AUDIT AGENCY,0511-AUDITING,90%,86%
1,DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY,1701-GENERAL EDUCATION AND TRAINING,59%,98%
2,DD25-DEFENSE LEGAL SERVICES AGENCY,0905-GENERAL ATTORNEY,70%,23%
3,DD48-DEFENSE HUMAN RESOURCES ACTIVITY,0301-MISCELLANEOUS ADMINISTRATION AND PROGRAM,80%,42%
4,DD65-PENTAGON FORCE PROTECTION AGENCY,0083-POLICE,70%,73%


#### Overlap Between Those Groups

In [18]:
overlap=putTogetherSpecializationOverlap(ddDataCopy, .5, .5)
overlap

,Agency,Occupation,Percent at Agency,Percent Overall
0,DD10-DEFENSE CONTRACT AUDIT AGENCY,0511-AUDITING,90%,86%
1,DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY,1701-GENERAL EDUCATION AND TRAINING,59%,98%
2,DD65-PENTAGON FORCE PROTECTION AGENCY,0083-POLICE,70%,73%


### Are Other Agencies Doing 'Owned' Functions?

In [19]:
for occ in overlap["Occupation"].unique():
    print(occ)
    print(ddDataCopy.loc[ddDataCopy['OCCT']==occ][['AGYSUBT', 'OCCT']].groupby(['AGYSUBT']).count().sort_values("OCCT", ascending=False))
    print()

0511-AUDITING
                                                OCCT
AGYSUBT                                             
DD10-DEFENSE CONTRACT AUDIT AGENCY              4398
DD26-OFFICE OF THE INSPECTOR GENERAL             613
DD35-DEFENSE FINANCE AND ACCOUNTING SERVICE       58
DD07-DEFENSE LOGISTICS AGENCY                     21
DD04-DEFENSE INFORMATION SYSTEMS AGENCY           14
DD27-MISSILE DEFENSE AGENCY                       14
DD34-DEFENSE COMMISSARY AGENCY                     8
DD63-DEFENSE CONTRACT MANAGEMENT AGENCY            7
DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF     2
DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY      2
DD53-DEFENSE POW/MIA ACCOUNTING AGENCY             1
DD81-DEFENSE ACQUISITION UNIVERSITY                1

1701-GENERAL EDUCATION AND TRAINING
                                                     OCCT
AGYSUBT                                                  
DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY       10749
DD69-NATIONAL DEFENSE UNIVERSITY 

### Measures of Similarity Across Agencies In Terms of Overall Occupational Profile

In [20]:
dfCosine=cosinSimAcrossAgencies(ddDataCopy, 'OCCT')
dfCosine.head(10)

,Org1,Org2,Similarity Index
0,DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF,DD01-OFFICE OF THE SECRETARY OF DEFENSE,0.961896
315,DD68-DEPARTMENT OF DEFENSE TEST RESOURCE MANAG...,DD27-MISSILE DEFENSE AGENCY,0.943637
172,DD48-DEFENSE HUMAN RESOURCES ACTIVITY,DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF,0.913131
209,DD53-DEFENSE POW/MIA ACCOUNTING AGENCY,DD48-DEFENSE HUMAN RESOURCES ACTIVITY,0.887822
83,DD25-DEFENSE LEGAL SERVICES AGENCY,DD08-U.S. COURT OF APPEALS FOR THE ARMED FORCES,0.886958
171,DD48-DEFENSE HUMAN RESOURCES ACTIVITY,DD01-OFFICE OF THE SECRETARY OF DEFENSE,0.875836
380,DD74-DEFENSE TECHNICAL INFORMATION CENTER,DD04-DEFENSE INFORMATION SYSTEMS AGENCY,0.868480
191,DD53-DEFENSE POW/MIA ACCOUNTING AGENCY,DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF,0.853985
250,DD61-DEFENSE THREAT REDUCTION AGENCY,DD48-DEFENSE HUMAN RESOURCES ACTIVITY,0.842478
232,DD61-DEFENSE THREAT REDUCTION AGENCY,DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF,0.837979


### Looking At Two Most Similar Agencies On This Metric

In [21]:
ddDataCopy.loc[ddDataCopy['AGYSUBT']=="DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF"]['OCCT'].value_counts().head()

0301-MISCELLANEOUS ADMINISTRATION AND PROGRAM    348
0343-MANAGEMENT AND PROGRAM ANALYSIS             165
1515-OPERATIONS RESEARCH                          47
2210-INFORMATION TECHNOLOGY MANAGEMENT            44
0501-FINANCIAL ADMINISTRATION AND PROGRAM         20
Name: OCCT, dtype: int64

In [22]:

ddDataCopy.loc[ddDataCopy['AGYSUBT']=="DD01-OFFICE OF THE SECRETARY OF DEFENSE"]['OCCT'].value_counts().head()

0301-MISCELLANEOUS ADMINISTRATION AND PROGRAM    762
0343-MANAGEMENT AND PROGRAM ANALYSIS             364
0130-FOREIGN AFFAIRS                             212
1515-OPERATIONS RESEARCH                         129
0801-GENERAL ENGINEERING                          85
Name: OCCT, dtype: int64

### We're Going to Measure Similarity Geographically

In [23]:
dfCosine=cosinSimAcrossAgencies(ddDataCopy, 'LOCT')
dfCosine.head(10)

,Org1,Org2,Similarity Index
316,DD68-DEPARTMENT OF DEFENSE TEST RESOURCE MANAG...,DD29-DEFENSE TECHNOLOGY SECURITY ADMINISTRATION,1.000000
308,DD68-DEPARTMENT OF DEFENSE TEST RESOURCE MANAG...,DD13-DEFENSE ADVANCED RESEARCH PROJECTS AGENCY,0.999960
128,DD29-DEFENSE TECHNOLOGY SECURITY ADMINISTRATION,DD13-DEFENSE ADVANCED RESEARCH PROJECTS AGENCY,0.999960
403,DD74-DEFENSE TECHNICAL INFORMATION CENTER,DD68-DEPARTMENT OF DEFENSE TEST RESOURCE MANAG...,0.999952
394,DD74-DEFENSE TECHNICAL INFORMATION CENTER,DD29-DEFENSE TECHNOLOGY SECURITY ADMINISTRATION,0.999952
386,DD74-DEFENSE TECHNICAL INFORMATION CENTER,DD13-DEFENSE ADVANCED RESEARCH PROJECTS AGENCY,0.999912
121,DD29-DEFENSE TECHNOLOGY SECURITY ADMINISTRATION,DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF,0.999888
301,DD68-DEPARTMENT OF DEFENSE TEST RESOURCE MANAG...,DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF,0.999888
29,DD13-DEFENSE ADVANCED RESEARCH PROJECTS AGENCY,DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF,0.999872
379,DD74-DEFENSE TECHNICAL INFORMATION CENTER,DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF,0.999841


### Looking At Most Geographically Similar Agencies

In [24]:
ddDataCopy.loc[ddDataCopy['AGYSUBT']=="DD68-DEPARTMENT OF DEFENSE TEST RESOURCE MANAGEMENT CENTER"]['LOCT'].value_counts().head()

51-VIRGINIA    25
Name: LOCT, dtype: int64

In [25]:
ddDataCopy.loc[ddDataCopy['AGYSUBT']=="DD29-DEFENSE TECHNOLOGY SECURITY ADMINISTRATION"]['LOCT'].value_counts().head()

51-VIRGINIA    122
Name: LOCT, dtype: int64

## This Is The End, Below Is In Progress: Mapping and USAJobs

In [26]:
def genStateDict():
    states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
    return(states)

In [27]:
stateDictionary=genStateDict()
stateUpper = {}
for k, v in stateDictionary.items():
    stateUpper[v.upper()] = k
stateUpper
stateSeries = pd.Series(stateUpper) 
stateDF=stateSeries.to_frame()
stateDF.columns=['State Abbreviation']

### State Name to Abbreviation

In [28]:
ddDataCopy['State Name']=ddDataCopy['LOCT'].str.split("-").str[1]

In [29]:
ddDataState=ddDataCopy.merge(stateDF, left_on='State Name', right_index=True, how='left')
#ddDataCopy['State Name'].value_counts()

### Good number of succesful merges

In [30]:
len(ddDataState['State Abbreviation'].value_counts())

53

### Construct State-Level Data

In [31]:
colors = ["#F1EEF6", "#D4B9DA", "#C994C7", "#DF65B0", "#DD1C77", "#980043"]


stateDF=ddDataState[['State Abbreviation', 'LOCT']].groupby(['State Abbreviation']).count().reset_index()
stateDF.columns=['State', 'Count']
stateDF.head()
biggestNum=stateDF['Count'].max()
biggestNum
biggestNum/len(colors)

4499.166666666667

In [32]:
EXCLUDED = ("ak", "hi", "pr", "gu", "vi", "mp", "as")

state_xs = [states[code]["lons"] for code in states]
state_ys = [states[code]["lats"] for code in states]


state_colors = []
for state_id in states:
    try:
        rate = stateDF.loc[stateDF['State']==state_id]['Count']
        idx = int(rate/(biggestNum/len(colors))-1)
        state_colors.append(colors[idx])
    except KeyError:
        state_colors.append("black")

p = figure(title="DoD Employees by State", toolbar_location="left",
           plot_width=800, plot_height=500)

p.patches(state_xs, state_ys, fill_color=state_colors,
          line_color="#884444", line_width=2, line_alpha=0.3)

show(p)

### Let's Scrape Some USAJOBS Data


In [33]:
listOfPossibleAgencyNumbers=list(range(0, 101))

dfAllDod=pd.DataFrame()
for num in listOfPossibleAgencyNumbers:
    try:
        df_current=current_search(num)
        dfAllDod = pd.concat([dfAllDod, df_current])
        if len(df_current)>0:
            print(f' {num} is {len(df_current)}')
    except:
        pass


#range of DoD agencies is 0-100


C:\Users\admin\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


 10 is 11
 11 is 18


C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


 12 is 2
 13 is 1
 16 is 94
 21 is 7
 25 is 3
 26 is 5
 27 is 16
 34 is 61
 35 is 43
 48 is 7
 60 is 51
 61 is 25
 63 is 109
 65 is 4
 69 is 2
 71 is 4
 74 is 2
 80 is 8
 81 is 4


In [34]:
dfAllDod['DepartmentName'].value_counts()
dfAllDod['OrganizationName'].value_counts()

Defense Contract Management Agency           109
Department of Defense Education Activity      94
Defense Commissary Agency                     61
Defense Health Agency                         51
Defense Finance and Accounting Service        43
Defense Threat Reduction Agency               25
National Geospatial-Intelligence Agency       18
Missile Defense Agency                        16
Defense Contract Audit Agency                 11
Defense Media Activity                         8
Defense Human Resources Activity               7
Washington Headquarters Services               7
Office of Inspector General                    5
Defense Microelectronics Activity              4
Pentagon Force Protection Agency               4
Defense Acquisition University                 4
Defense Legal Services Agency                  3
Defense Technical Information Center           2
National Defense University                    2
Defense Security Service                       2
Defense Advanced Res

In [35]:
for column in dfAllDod:
    print(column)
    print(dfAllDod[column].iloc[0])
    print("")


ApplicationCloseDate
2019-06-28

ApplyURI
['https://www.usajobs.gov:443/GetJob/ViewDetails/535929700?PostingChannelID=RESTAPI']

DepartmentName
Department of Defense

JobCategory
[{'Name': 'Auditing', 'Code': '0511'}]

JobGrade
[{'Code': 'GS'}]

OrganizationName
Defense Contract Audit Agency

PositionEndDate
2019-06-28

PositionFormattedDescription
[{'Label': 'Dynamic Teaser', 'LabelDescription': 'Hit highlighting for keyword searches.'}]

PositionID
DCAA-WE-19-10514666

PositionLocation
[{'LocationName': 'La Palma, California', 'CountryCode': 'United States', 'CountrySubDivisionCode': 'California', 'CityName': 'La Palma, California', 'Longitude': -118.037285, 'Latitude': 33.8483849}]

PositionLocationDisplay
La Palma, California

PositionOfferingType
[{'Name': 'Permanent', 'Code': '15317'}]

PositionRemuneration
[{'MinimumRange': '100820.00', 'MaximumRange': '131064.00', 'RateIntervalCode': 'Per Year'}]

PositionSchedule
[{'Name': 'Full-Time', 'Code': '1'}]

PositionStartDate
2019-06-

### Looking At DLA

In [36]:
ddDataState['OCCT_#']=ddDataState['OCCT'].str[0:4].astype(str)
dataWithOcc=ddDataState.merge(dfOccs, left_on='OCCT_#', right_on='Occupation Code')
dataDLA=dataWithOcc.loc[dataWithOcc['AGYSUBT']=="DD07-DEFENSE LOGISTICS AGENCY"]


missingOccsDoD=[i for i in dfOccs['Occupation Code'].unique() if i not in dataWithOcc['OCCT_#'].unique()]
missingOccsDLA=[i for i in dfOccs['Occupation Code'].unique() if i not in dataDLA['OCCT_#'].unique()]
#we're missing a lot of occupations but maybe they just don't have them at DLA


#let's test


### The Occupational Codes Related to EBOs and Not In DoD

In [37]:
dfOccs.loc[dfOccs['Occupation Code'].isin(missingOccsDoD)]

,EBO,Occupation Series,Occupation Code,Series Description
6,Human Resources,Mediation Series,0241,This series covers all positions that involve ...
7,Human Resources,Apprenticeship and Training Series,0243,This series includes positions that involve th...
8,Human Resources,Labor Management Relations Examining Series,0244,This series includes positions that involve th...
16,Financial Management,Internal Revenue Agent Series,0512,This series covers positions that determine or...
18,Financial Management,Tax Specialist Series,0526,"This series covers positions that determine, s..."
26,Financial Management,Tax Examining Series,0592,This series covers all positions the paramount...


### The Occupational Codes Related to EBOs and Not In DLA

In [38]:
dfOccs.loc[dfOccs['Occupation Code'].isin(missingOccsDLA)]

,EBO,Occupation Series,Occupation Code,Series Description
1,Acquisition and Procurement,Telecommunications Series,0390,This occupation includes positions that involv...
6,Human Resources,Mediation Series,0241,This series covers all positions that involve ...
7,Human Resources,Apprenticeship and Training Series,0243,This series includes positions that involve th...
8,Human Resources,Labor Management Relations Examining Series,0244,This series includes positions that involve th...
9,Human Resources,Human Resources Management Student Trainee Series,0299,See the series definition for the General Stud...
16,Financial Management,Internal Revenue Agent Series,0512,This series covers positions that determine or...
18,Financial Management,Tax Specialist Series,0526,"This series covers positions that determine, s..."
19,Financial Management,Cash Processing Series,0530,This series covers positions that involve cler...
20,Financial Management,Voucher Examining Series,0540,This series covers all classes of positions th...
21,Financial Management,Civilian Pay Series,0544,This series covers positions concerned with th...


### What We Do See In DLA by EBO

In [39]:
dfOccs.loc[~dfOccs['Occupation Code'].isin(missingOccsDLA)]['EBO'].value_counts()

Financial Management           8
Supply and Logistics           8
Real Property                  4
Human Resources                3
Acquisition and Procurement    3
Information Technology         2
Name: EBO, dtype: int64

In [40]:
### I want to get a data frame where the index is states and the variables are numbers of employees grouped by EBO by agency
### or we can a dictionary with name of EBO?, and the value is the dataframe content?
def takeEBOProductByAgencyNumbers(df, EBO):
    dfEBO=df.loc[df['EBO']==EBO].loc[~df['State Abbreviation'].isnull()]
    dfGrouped=pd.pivot_table(dfEBO, values='EBO', index=['State Abbreviation'], columns=['AGYSUBT'], aggfunc='count')
    dfGrouped=dfGrouped.reindex([i for i in  genStateDict().keys()])
    return(dfGrouped)

dictionaryOfEBOsAndStates={}
for EBO in dfOccs['EBO'].unique():
    dictionaryOfEBOsAndStates[EBO]=takeEBOProductByAgencyNumbers(dataWithOcc, EBO)
    print(EBO)
    print(dictionaryOfEBOsAndStates[EBO]['DD07-DEFENSE LOGISTICS AGENCY'].sum())

Acquisition and Procurement
3864.0
Human Resources
718.0
Financial Management
641.0
Information Technology
1306.0
Real Property
96.0
Supply and Logistics
4203.0


In [41]:
for column in dictionaryOfEBOsAndStates['Acquisition and Procurement'].columns:
    print(column, dictionaryOfEBOsAndStates['Acquisition and Procurement'][column].sum())

DD01-OFFICE OF THE SECRETARY OF DEFENSE 29.0
DD02-ORGANIZATION OF THE JOINT CHIEFS OF STAFF 5.0
DD04-DEFENSE INFORMATION SYSTEMS AGENCY 358.0
DD06-DEFENSE SECURITY COOPERATION AGENCY 11.0
DD07-DEFENSE LOGISTICS AGENCY 3864.0
DD10-DEFENSE CONTRACT AUDIT AGENCY 8.0
DD12-DEFENSE SECURITY SERVICE 17.0
DD13-DEFENSE ADVANCED RESEARCH PROJECTS AGENCY 16.0
DD15-UNIFORMED SERVICES UNIVERSITY OF THE HEALTH SCIENCES 18.0
DD16-DEPARTMENT OF DEFENSE EDUCATION ACTIVITY 44.0
DD21-WASHINGTON HEADQUARTERS SERVICES 115.0
DD26-OFFICE OF THE INSPECTOR GENERAL 1.0
DD27-MISSILE DEFENSE AGENCY 242.0
DD34-DEFENSE COMMISSARY AGENCY 115.0
DD35-DEFENSE FINANCE AND ACCOUNTING SERVICE 65.0
DD48-DEFENSE HUMAN RESOURCES ACTIVITY 32.0
DD53-DEFENSE POW/MIA ACCOUNTING AGENCY 2.0
DD60-DEFENSE HEALTH AGENCY 102.0
DD61-DEFENSE THREAT REDUCTION AGENCY 80.0
DD63-DEFENSE CONTRACT MANAGEMENT AGENCY 3079.0
DD65-PENTAGON FORCE PROTECTION AGENCY 18.0
DD69-NATIONAL DEFENSE UNIVERSITY 15.0
DD71-DEFENSE MICROELECTRONICS ACTIVITY 22

In [42]:
#let's look just at acq and proc state data
stateAcq=dataWithOcc.loc[dataWithOcc['EBO']=='Acquisition and Procurement'].loc[~dataWithOcc['State Abbreviation'].isnull()]
dfCosine=cosinSimAcrossAgencies(stateAcq, 'State Abbreviation')
dfCosine.loc[dfCosine['Org1']=='DD07-DEFENSE LOGISTICS AGENCY']
dfCosine.loc[dfCosine['Org2']=='DD07-DEFENSE LOGISTICS AGENCY']

,Org1,Org2,Similarity Index
280,DD81-DEFENSE ACQUISITION UNIVERSITY,DD07-DEFENSE LOGISTICS AGENCY,0.639991
49,DD21-WASHINGTON HEADQUARTERS SERVICES,DD07-DEFENSE LOGISTICS AGENCY,0.636599
194,DD65-PENTAGON FORCE PROTECTION AGENCY,DD07-DEFENSE LOGISTICS AGENCY,0.636480
19,DD12-DEFENSE SECURITY SERVICE,DD07-DEFENSE LOGISTICS AGENCY,0.636480
25,DD13-DEFENSE ADVANCED RESEARCH PROJECTS AGENCY,DD07-DEFENSE LOGISTICS AGENCY,0.636480
109,DD48-DEFENSE HUMAN RESOURCES ACTIVITY,DD07-DEFENSE LOGISTICS AGENCY,0.636480
59,DD26-OFFICE OF THE INSPECTOR GENERAL,DD07-DEFENSE LOGISTICS AGENCY,0.636480
157,DD61-DEFENSE THREAT REDUCTION AGENCY,DD07-DEFENSE LOGISTICS AGENCY,0.635600
304,DD82-NATIONAL RECONNAISSANCE OFFICE,DD07-DEFENSE LOGISTICS AGENCY,0.635340
82,DD34-DEFENSE COMMISSARY AGENCY,DD07-DEFENSE LOGISTICS AGENCY,0.633927


In [43]:
dictionaryOfEBOsAndStates['Acquisition and Procurement'][['DD07-DEFENSE LOGISTICS AGENCY','DD63-DEFENSE CONTRACT MANAGEMENT AGENCY']]

AGYSUBT,DD07-DEFENSE LOGISTICS AGENCY,DD63-DEFENSE CONTRACT MANAGEMENT AGENCY
State Abbreviation,,
AK,NaN,NaN
AL,76.0,87.0
AR,NaN,NaN
AS,NaN,NaN
AZ,NaN,144.0
CA,10.0,379.0
CO,NaN,97.0
CT,NaN,129.0
DC,NaN,2.0


In [68]:
def takeSeriesMakeUSMap(mySeries, myStringSide, myStringTitle):
    import plotly.plotly as py
    import plotly

    plotly.tools.set_credentials_file(username='abhaddad', api_key='ei48LLb6cBsvvAZ9AZwD')
    scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]
    data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = mySeries.index,
        z = mySeries.astype(float),
        locationmode = 'USA-states',
        text = mySeries.index,
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            )
        ),
        colorbar = dict(
            title = myStringSide
        )
    ) ]

    layout = dict(
        title = myStringTitle,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)',
        ),
    )

    fig = dict( data=data, layout=layout )
    titleOfFile=r"C:\Users\admin\Documents\Python Scripts\personnel\personnelData\Scripts\images" + myStringTitle + ".png"
    print(titleOfFile)
    pio.write_image(fig, titleOfFile )
    url = py.plot( fig, filename='d3-cloropleth-map' )
    return(url)

In [69]:
mySeries=dictionaryOfEBOsAndStates['Acquisition and Procurement']['DD07-DEFENSE LOGISTICS AGENCY'].fillna(0)

In [72]:
import plotly.io as pio
listOfEBOs=list(dictionaryOfEBOsAndStates.keys())
for EBO in listOfEBOs:
    mySeries=dictionaryOfEBOsAndStates[EBO]['DD07-DEFENSE LOGISTICS AGENCY'].fillna(0)
    print(mySeries.sum())
    takeSeriesMakeUSMap(mySeries, "Values", f"DLA Workforce - {EBO}")
    

3864.0
C:\Users\admin\Documents\Python Scripts\personnel\personnelData\Scripts\imagesDLA Workforce - Acquisition and Procurement.png
718.0
C:\Users\admin\Documents\Python Scripts\personnel\personnelData\Scripts\imagesDLA Workforce - Human Resources.png
641.0
C:\Users\admin\Documents\Python Scripts\personnel\personnelData\Scripts\imagesDLA Workforce - Financial Management.png
1306.0
C:\Users\admin\Documents\Python Scripts\personnel\personnelData\Scripts\imagesDLA Workforce - Information Technology.png
96.0
C:\Users\admin\Documents\Python Scripts\personnel\personnelData\Scripts\imagesDLA Workforce - Real Property.png
4203.0
C:\Users\admin\Documents\Python Scripts\personnel\personnelData\Scripts\imagesDLA Workforce - Supply and Logistics.png


### Percent International

In [89]:
dataDLA=dataWithOcc.loc[dataWithOcc['AGYSUBT']=='DD07-DEFENSE LOGISTICS AGENCY']
dataDLA.loc[dataDLA['EBO']=="Supply and Logistics"]['LOCT'].value_counts(normalize=True)
#pd.crosstab(dataDLA['LOCT'], dataDLA['EBO'])
#dataDLA.columns
for EBO in dataDLA['EBO'].unique():
    print(EBO)
    print(1-dataDLA.loc[dataDLA['EBO']==EBO]['State Abbreviation'].count()/len(dataDLA.loc[dataDLA['EBO']==EBO]))

Financial Management
0.0031104199066873672
Acquisition and Procurement
0.003866976024748614
Information Technology
0.028996282527881
Human Resources
0.004160887656033285
Supply and Logistics
0.02640722724113964
Real Property
0.040000000000000036
